In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def extractInfo(data,nSamples):
    """Extract relevant information from the matlab data object"""
    
    channelNames = [channel[0] for channel in data["nfo"][0][0][2][0]]

    #Extracting Sampling Rate
    sRate = data["nfo"][0][0][0][0][0]

    # Extracting class labels
    classLabels = list(map(lambda x : x[0], data["nfo"][0][0][1][0]))

    #Extracting event onset data
    eventOnsets = data["mrk"][0][0][0]


    #Extracting event code data
    eventCodes = data["mrk"][0][0][1]

    #labels for each eeg data sample
    labels  = np.zeros((1,nSamples),dtype=int)

    # Set labels positions to event codes using eventOnsets as indexes
    # This ensure that data samples that arent associated with any event have a target value of zero

    labels[0,eventOnsets] = eventCodes

    return channelNames,sRate,classLabels,eventOnsets,eventCodes,labels


In [3]:
def getTrials(labels, uniqueEventCodes, trialWindow,eeg,eventCodes_train_test,eventOnsets_train_test,sRate, nChannels,selectedChannels=None):

    """Extract trials from continuous EEG data 
    
    Paramters
    ---------
    labels - An Array containing the class labels (left and right)

    uniqueEventCodes - Event codes corresponding to a particular label (-1 and 1)

    trialWindow - An array containing values representing the trial window. This array is added 
                  to a given event onset time sample point to extract the effective trial window
    
    eeg - 2D array of shape channel x samples

    eventOnsets_train_test - An array containing the event onset from which to extract trials 
    
    eventCodes_train_test - An array containing the event codes corresponding to the given event onsets

    nChannels - The number of channels in the data

    selectedChannels - An array containing specific channels from which to extract data


    Returns
    -------
    trials - a dictionary object containing extracted trials for the given motor imagery classes
    
    """

    #In order to obtain the appropriate trials for this dataset, we need to define a time window for epoching the data.

    #The NFFT param value for the PSD function must be exactly half of the length of the
    # sample window + 1
    #It seems that increasing the length of the time window increases the classification accuracy but only up on until 
    # time samples
    #Create visual representaion for this.
    #It may be useful to show also the effect of shifting the start time of the window 0.0, 0.1, 0.2 etc

    trials = {}

    idleStateTrials = {}

    
    
    for cls, code in zip(labels,uniqueEventCodes):

        #Get all event onsets for the particular class
        # Create a filter array i.e return an array with True
        # values at all indeces where the eventCode = code
        # and False otherwise

        filter_arr = eventCodes_train_test == code

        # use arr to filter event onsest specific to this class
        clsOnsets = eventOnsets_train_test[filter_arr]
        
        #Allocate memmory for trial
        trials[cls] = np.zeros(
            (nChannels if selectedChannels is None else len(selectedChannels), len(clsOnsets), len(trialWindow)))
        
        idleStateTrials[cls] = np.zeros(
            (nChannels if selectedChannels is None else len(selectedChannels), len(clsOnsets), len(trialWindow)))
        
        
        #Extract trials for the class

        #make third class? Yes
        
        for i, onset in enumerate(clsOnsets):
            #For all 59 channels extract class trials of size onset + win from
            # Each onsent represents the start time of an external cue
            #Each row is a list of the values for each channel of the 59 channels recorded
            #within the time window of interest
                
            selectedWindow = eeg[:,onset+trialWindow] 
            # print(f"selected window shape: {selectedWindow.shape}")

            # initialize selectedIdleState with zeros
            selectedIdleState = np.zeros((nChannels, 4*sRate))

            start_index = onset + (4 * sRate)
            end_index = start_index + trialWindow.shape[0]

            if end_index > eeg.shape[1]:
                # if the end index is out of range for eeg, extract the valid portion of eeg
                valid_eeg = eeg[:, start_index:]
                # copy the valid portion of eeg into the corresponding portion of selectedIdleState
                selectedIdleState[:, :valid_eeg.shape[1]] = valid_eeg
            else:
                # if the slice is entirely within the bounds of eeg, extract it directly
                selectedIdleState[:, :trialWindow.shape[0]] = eeg[:, start_index:end_index]
                
            trials.get(cls)[:,i,:] = selectedWindow if selectedChannels is None else selectedWindow[selectedChannels,:]
            idleStateTrials.get(cls)[:,i,:] = selectedIdleState if selectedChannels is None else selectedIdleState[selectedChannels,:]
            
    
    return trials, idleStateTrials


In [4]:
from scipy import signal

def getFilteredTrials(trials, trialWin,nChannels,sRate,b=None,a=None,selectedChannels=None, classLabels=None):
    """Extract filtered trials 
    
    Parameters
    ----------
    trials - a dictionary object containing extracted trials the given motor imagery classes

    trialWin - An array containing values representing the trial window. This array is added 
                  to a given event onset time sample point to extract the effective trial window
    
    nChannels - The number of channels in the data

    sRate - The sampling rate

    b - numerator (b)  coefficients of an iirfilter

    a - denominator (a) coefficients of an iirfilter

    selectedChannels - An array containing specific channels from which to extract data.

    Returns
    -------
    trials_filt: A dictionary object contatining filtered trials for the given motor imagery classes
    
    """
    
    trials_filt = {}

    def bandPass(trial_lr,lowcut,highcut,fs):
        nonlocal b, a
        """Bandpass filter 

        Parameters
        ----------
        trial_lr: A 3D ndarray of shape (channels x trials x time) which contains the trials per channels

        lowcut: Lower frequency bound in Hz

        highcut: Higher frequency bound in Hz
        
        fs: Sampling frequency

        Returns
        -------
        trials_filt: A 3D ndarray of shape (channels x trials x time) which contains the bandpass filtered trials per channels
        """
        nqfreq = 0.5*fs
        
        if b is None and a is None:
            b , a = signal.iirfilter(4,[lowcut/nqfreq,highcut/nqfreq])

        nTrials = trial_lr.shape[1]
        filt_trials = np.zeros((nChannels if selectedChannels is None else len(selectedChannels),nTrials,len(trialWin)))


        for t in range(nTrials):
            filt_trials[:,t,:] = signal.filtfilt(b,a,trial_lr[:,t,:],axis=1)

        return filt_trials
    
    cl1, cl2 = classLabels

    trials_filt[cl1] = bandPass(trials[cl1],8,12,sRate)
    trials_filt[cl2] = bandPass(trials[cl2],8,12,sRate)

    return trials_filt


In [5]:
from matplotlib import mlab

def psd(trials, trialWindow, nChannels, sRate):
    """
    Parameters
    ----------
    trials: A 3D ndarray of shape (channels x trials x time)

    Returns
    -------
    trials_PSD: A 3D ndarray of shape (channels x trials x PSD) the PSD for each trial
    """

    nTrials = trials.shape[1]
    trials_PSD = np.zeros((nChannels,nTrials, int(len(trialWindow)/2)+1)) #Why?

    for trial in range(nTrials):
        for ch in range(nChannels):
            #Calculate the PSD
            
            (PSD, freqs) = mlab.psd(trials[ch, trial , :], NFFT=len(trialWindow), Fs=sRate)
            trials_PSD[ch, trial, :] = PSD

    return trials_PSD, freqs


In [6]:
def plot_psd(trial_PSDs,freqs,channel_IDXs,ymax,labels=None):

    """ Plot the mean Power Spectral Density of left and right hand signals from 
        all trials at the given electrode channels """

    plt.figure(figsize=(12,5))
    nChans = len(channel_IDXs)
    nRows = int(np.ceil(nChans/3))
    nCols = min(3,nChans)

    #for channels in channel indexes
    for i,ch in enumerate(channel_IDXs):
        plt.subplot(nRows,nCols,i+1)

        for cls in trial_PSDs.keys():
            plt.plot(freqs,np.mean(trial_PSDs[cls][ch,:,:],axis=0),label=cls)

        #plt.fill_betweenx(np.mean(trial_PSDs["left"][ch,:,:],axis=0), 8, 12, color="green", alpha=0.2)

        plt.xlabel("Frequency in Hz")
        
        plt.xlim(1,30)
        plt.grid()
        plt.ylim(0,ymax)

        if labels is not None:
            plt.legend(labels)
        else:
            plt.legend()

    plt.tight_layout()


In [7]:
#Following the standard bci classification paradigm outlined in "The non-invasive Berlin Brain–Computer Interface: Fast acquisition of effective performance in untrained subjects"
#Benjamin Blankertz,a,⁎ Guido Dornhege,a Matthias Krauledat,a,b Klaus-Robert Müller, and Gabriel Curio

def logvar(trials):
    #trials has a shape of 59 x 100 x 200
    return np.log(np.var(trials,axis=2))
    #calculate variance along the sample (time sample) axis
    #then calculate the log of the result

    # Since VARIANCE of band-pass filtered signals
    # is equal to band-power, CSP analysis is applied
    # to approximately band-pass filtered signals in order
    # to obtain an effective discrimination of mental states
    # that are characterized by ERD/ERS effects ref:Optimizing Spatial filters for Robust EEG Single-Trial Analysis

    # The log of the variance can be useful for data that exhibits exponential or power-law relationships,
    # as it can help to compress the range of the data and make it easier to visualize and analyze.

    # For example, consider a set of spatial filters with variances[1, 10, 100]. The log of the variances
    # would be[0, 1, 2], which has a smaller range than the original data. This can be useful for data that
    # has a wide range of values, as it can make it easier to visualize and analyze the data.


In [8]:
def plot_logvar(trials, nChannels, classLabels):
    """ Plot the mean log-var (logarithm of the variance) """
    plt.figure(figsize=(12,5))

    x0 = np.arange(nChannels)
    x1 = np.arange(nChannels) + 0.4

    cl1,cl2 = classLabels

    y0 = np.mean(trials[cl1],axis=1)
    y1 = np.mean(trials[cl2],axis=1)

    #axis 1 refers to the axis at position 1 in the dimension tuple
    #in this case, the dimension tuple of trials["left"] or trials["right"]
    # is (59,100)
    #Hence the mean is calculated along the axis with 100 values
    #Leaving a vector of shape (59,)


    plt.bar(x0,y0, width=0.5, color="b")
    plt.bar(x1,y1, width=0.4, color="r")

    plt.xlim(-0.5,nChannels+0.5)

    plt.gca().yaxis.grid(True)
    plt.title("log-var of each channel")
    plt.xlabel("channels")
    plt.ylabel("log-var")
    plt.legend(classLabels)

    #A plot of the log of the variance of the CSP transformed data can be useful for identifying patterns or
    # trends in the data that may be indicative of differences between the two classes of signals.
    # For example, if the log of the variance of the CSP transformed data is higher for one class than the
    # other, it could indicate that the filters for that class are more discriminative.

    # To choose the subset of filters that are most discriminative between the two classes of signals,
    # it may be useful to plot the log of the variance of the CSP transformed data and examine the patterns
    # or trends that emerge. The filters with the highest variance could be selected as the most discriminative,
    # as they may contain the most information about the differences between the two classes.

    #In the case below, it seems that the log of the variance of spatial filter 0 (column 0) is at its highest for
    #left hand signals and its lowest for right hand singals, whereas, spatial filter 59, has its log-var at its highest
    #right hand signals and its lowest for left hand signals. Hence, for each signal window (shape 59 x 1 x 200)
    # We can extract the two most relevant spatial filters to determine whether it represents left or right.

    


In [9]:
from numpy import linalg

def cov(trials):
    """Calculate the covariance for each trial and return their average
    
    Parameters
    ----------
    trials - Array (channels x trials x samples) 

    Returns
    -------
    A covariance matrix containing the mean values of all covariance matrices generated from trials
    """
    ntrials = trials.shape[1]

    #Select all 59 channels (59 rows), then select the ith trial
    #for each of those 59 rows along with all the columns (values) for that
    # trial (resulting in a 59 x 400 matrix) and calculate the covariance matrix

    covs = [np.cov(trials[:,i,:]) for i in range(ntrials)]

    # covs consists of nTrials x 59 x 59 matrices
    #Since covariance is calculated by the dot product of X and X.T
    #Where X.T is the transpose of X [(59 x 400).dot((400 x 59))]

    #print(np.array(covs).shape)

    return np.mean(covs, axis=0)


In [10]:
def whitening(sigma):
    """Calculate a whitening matrix for covariance matrix sigma.

    Paramters
    ---------
    sigma - A covariance matrix of shape N x N, where N is the number of channels for a given
            trial. 

    Returns
    --------
    The whitened covariance matrix

    """
    #In singular value decomposition(SVD), a matrix is decomposed into the product of
    # three matrices: a left singular matrix, a diagonal matrix, and a right singular matrix.
    # The diagonal matrix, called the singular value matrix, contains the singular values of the
    # original matrix. The left and right singular matrices contain the left and right singular
    # vectors, respectively.

    # A whitening matrix is a diagonal matrix that is used to transform the singular value matrix
    # so that it has the identity matrix as its diagonal. This is done by dividing each element on the
    # diagonal of the singular value matrix by the square root of the corresponding singular value.

    # The whitening matrix is useful in SVD because it can be used to decorrelate the singular vectors,
    # which can simplify certain computations and make it easier to interpret the results of
    # the decomposition. For example, in dimensionality reduction, whitening the singular value matrix
    # can help to remove some of the redundancy in the data and make it easier to identify the underlying
    # structure of the data.

    U, l, _ = linalg.svd(sigma) # l is a vector of singular values U is the left singular matrix
    return U.dot(np.diag(l ** -0.5))

In [11]:
import scipy

def csp(trials_r, trials_l):
    """Calculate the CSP transformation matrix W

    Paramters
    ---------
    trials_r - Array(channels x trials x samples) containing right hand movement trials
    trials_l - Array(channels x trials x samples) containing left hand movement trials
    
    Returns
    -------
    Mixing matrix W
    """
    cov_l = cov(trials_l)
    cov_r = cov(trials_r)

    # apply whitening to covariance matrices
    P = whitening(cov_l + cov_r)
    cov_l = P @ cov_l @ P.T
    cov_r = P @ cov_r @ P.T
    
    
    # According to [Filter bank common spatial pattern algorithm on BCI competition IV Datasets 2a and 2b]
    # W can be calculated by W = eig(S1, S1 + S2),where W, S1, and S2 here represents W b, Σb,1, and Σb,2 respectively
    # Σb,1 and Σb,2 are estimates of the covariance matrices of the band-pass filtered EEG measurements of the respective motor imagery action (left, right)

    _,W = scipy.linalg.eigh(cov_l, cov_l + cov_r)

    # project data onto CSP components
    W = P.T @ W

    return W


In [12]:
def apply_mix(W, trials, trialWin, nChannels, selectedChannels=None):
    """Apply a decomposition matrix to each trial (basically multiply W with the EEG signal matrix)"""
    ntrials = trials.shape[1]


    trials_csp = np.zeros((nChannels if selectedChannels is None else len(selectedChannels) , ntrials, len(trialWin)))


    for i in range(ntrials):
        trials_csp[:,i,:] = W.T.dot(trials[:,i,:])

    return trials_csp


In [13]:
def scatter(cl1,cl2,classLabels):
    """ Display scatter plot of the distribution left and right motor imagery trials"""
    plt.figure()
    plt.scatter(cl1[0,:],cl1[-1,:],color="b")
    plt.scatter(cl2[0,:],cl2[-1,:],color="r")
    plt.xlabel("First Component")
    plt.ylabel("Last Component")
    plt.legend(classLabels)


In [14]:
import pandas as pd

def label_data(data,label,foldNum):
    """
    Parameters
    ----------
    data - An array of dimensions observation x features
    label the desired label

    Returns
    -------
    newData - An dataframe of with columns F1...Fn and label
    """

    nTrainSamples = data.shape[0]
    nFeatures = data.shape[1]

    featureColumns = [f"F{x}" for x in range(1,nFeatures+1)]

    newData = pd.DataFrame(data,columns=featureColumns)

    newData["Labels"] = np.array([label for x in range(nTrainSamples)])

    # Apply fold num
    newData["Fold"] = np.random.randint(1,foldNum+1,nTrainSamples)

    return newData


In [15]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

from collections import defaultdict

def tuneModelHyperParams(train_X, train_y, test_X, test_y):

    metric_model_performance = defaultdict(dict)

    for scoring in ["accuracy","roc_auc","f1"]:

        ################################### SVC ##########################################################

        param_grid = {'C': [0.0001,0.001,0.01,1,10,100,1000],
                    'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                    'kernel':["rbf","linear"],
                    'class_weight':['balanced', None]}

        gridSVM = GridSearchCV(estimator=SVC(probability=True), param_grid=param_grid,scoring=scoring, cv=5)

        gridSVM.fit(train_X, train_y)

        preds = gridSVM.best_estimator_.predict(test_X)

        metric_model_performance[scoring]["SVC"] = gridSVM.best_estimator_, sum(preds == test_y)/len(test_y)


        ################################### Logisitic Regression ##########################################

        param_grid = {'penalty': ['l1','l2'],
                      'C':list(np.logspace(-3,3)),
                      'solver': ['newton-cg', 'lbfgs', 'liblinear'],
                      'class_weight':['balanced', None]}

        # Create the grid search object
        gridLogReg = GridSearchCV(estimator=LogisticRegression(), param_grid=param_grid,scoring=scoring, cv=5)

        gridLogReg.fit(train_X,train_y)

        preds = gridLogReg.best_estimator_.predict(test_X)

        metric_model_performance[scoring]["LogReg"] = gridLogReg.best_estimator_, sum(preds == test_y)/len(test_y)


        ######################################### LDA ########################################################

        param_grid = {'solver': ['svd', 'lsqr', 'eigen'],'shrinkage': np.arange(0,1,0.1)}

        gridLDA = GridSearchCV(estimator=LDA(), param_grid=param_grid,scoring=scoring, cv=5)

        gridLDA.fit(train_X, train_y)

        preds = gridLDA.best_estimator_.predict(test_X)

        metric_model_performance[scoring]["LDA"] = gridLDA.best_estimator_, sum(preds == test_y)/len(test_y)

    return metric_model_performance


In [16]:
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids
from sklearn.model_selection import train_test_split
from collections import Counter

def featureExtraction(data, split_percentage, ExtractionType = None, b=None, a=None, selectedChannels=None, trialWinStart=0, trialWinEnd=4):

    """Extract csp features from training data"""
    
    eeg = data["cnt"].T 

    nChannels, nSamples = eeg.shape

    channelNames,sRate,classLabels,eventOnsets,eventCodes,labels = extractInfo(data,nSamples)

    cl1, cl2 = classLabels #left | right
    

    trialWindow = np.arange(int(trialWinStart*sRate),int(trialWinEnd*sRate))

    ####### RETURN MOTION INTENT STATE AND IDLE STATE TRIALS ########

    trials, idleStateTrials = getTrials(classLabels,
                        np.unique(eventCodes),
                        trialWindow,
                        eeg,
                        eventCodes,
                        eventOnsets,
                        sRate,
                        nChannels,
                        selectedChannels = selectedChannels)


    ######## RETURN FILTERED MOTION INTENT TRIALS AND IDLE STATE TRIALS#######
    

    filteredTrials = getFilteredTrials(trials,
                                       trialWindow,
                                       nChannels,
                                       sRate,
                                       selectedChannels=selectedChannels,
                                       b=b,
                                       a=a,
                                       classLabels=classLabels)
    

    filteredIdleTrials = getFilteredTrials(idleStateTrials,
                                           trialWindow,
                                           nChannels,
                                           sRate,
                                           selectedChannels=selectedChannels,
                                           b=b,
                                           a=a,
                                           classLabels=classLabels)
    

    ######### COMBINE IDLE TRIALS AND MOTION INTENT CLASSES INTO TWO SUPERCLASSES  #########
    
    concatFilteredIdleTrials = np.concatenate((filteredIdleTrials[cl1],filteredIdleTrials[cl2]),axis=1)
    concatFilteredMotionIntentTrials = np.concatenate((filteredTrials[cl1],filteredTrials[cl2]),axis=1)
    
    
    ##### CALCULTE A CSP TRANSORMATION MATRIX AND APPLY IT TO THE DATA USING THE IDLE STATE CLASS ########


    idle_MotionIntentTrainingData = {"idle": concatFilteredIdleTrials,
                                    "motion":concatFilteredMotionIntentTrials}
    
    W_idleMotionIntentTranformationMatrix = csp(idle_MotionIntentTrainingData["idle"],idle_MotionIntentTrainingData["motion"])

    idle_MotionIntentTrainingData = {
        "idle": apply_mix(W_idleMotionIntentTranformationMatrix, idle_MotionIntentTrainingData["idle"], trialWindow,nChannels,selectedChannels=selectedChannels),
        "motion":apply_mix(W_idleMotionIntentTranformationMatrix, idle_MotionIntentTrainingData["motion"], trialWindow,nChannels,selectedChannels=selectedChannels)
    }


    ##### CALCULTE A CSP TRANSORMATION MATRIX FOR ORIGINAL LEFT HAND IMAGERY AND RIGHT HAND IMAGERY AND APPLY IT TO THE DATA ########
    train = {cl1: filteredTrials[cl1],
            cl2: filteredTrials[cl2]}

    W = csp(train[cl2],train[cl1])

    train = {
    cl1 : apply_mix(W,train[cl1],trialWindow,nChannels,selectedChannels=selectedChannels),
    cl2 : apply_mix(W,train[cl2],trialWindow,nChannels,selectedChannels=selectedChannels)
    }

    ##### CALCULATE THE LOGVAR FOR BOTH TRANSFORMED SETS OF DATA #####

    idle_MotionIntentTrainingData["idle"] = logvar(idle_MotionIntentTrainingData["idle"])
    idle_MotionIntentTrainingData["motion"] = logvar(idle_MotionIntentTrainingData["motion"])

 
    train[cl1] = logvar(train[cl1])
    train[cl2] = logvar(train[cl2])


    #### CHOOSE THE IDLE COMPONENTS FOR EACH OF THE NEW CSP FEATURES FOR THE TWO TRANSFORMED SETS OF DATA #####
    comp = [0,-1]

    idle_MotionIntentTrainingData["idle"] = idle_MotionIntentTrainingData["idle"][comp,:]
    idle_MotionIntentTrainingData["motion"] = idle_MotionIntentTrainingData["motion"][comp,:] 


    train[cl1] = train[cl1][comp,:]
    train[cl2] = train[cl2][comp,:]


    #### LABEL DATA AND STORE IN A DATAFRAME FOR LATER PROCESSING ######

    train_idle_dataframe = label_data(idle_MotionIntentTrainingData["idle"].T,0,5)
    train_motionIntent_dataframe = label_data(idle_MotionIntentTrainingData["motion"].T,1,5)

    train_l_df = label_data(train[cl1].T,-1,5) 
    train_r_df = label_data(train[cl2].T,1,5)

    ###### CONCATE DATAFRAMES OF DIFFERENT CLASSES INTO ONE DATA FRAME #####

    train_idle_motionIntent_comb = pd.concat([train_idle_dataframe,train_motionIntent_dataframe])

    train_comb = pd.concat([train_l_df,train_r_df])

    nCols = len(train_comb.columns)

    if split_percentage == 1:
        if ExtractionType is None:

            X = train_comb.iloc[:,:nCols-2]
            y = train_comb.iloc[:,nCols-2]


            trainDataStore = {} #Stores different training data for each resampling method

            ######################################################## RESAMPLING #################################################################

            for name,resampler in [("RO",RandomOverSampler(random_state=42)),
                                    ("SMOTE",SMOTE(random_state=42)),
                                    ("RU",RandomUnderSampler(random_state=42)),
                                    ("CC",ClusterCentroids(random_state=42)),
                                    ("None",None)]:



                if not resampler:
                    train_comb = pd.DataFrame(X,columns=["F1","F2"])
                    train_comb["Labels"] = y

                    #store this instance of training data
                    trainDataStore[name] = train_comb.copy()
                    continue

                #Resample training data only
                train_comb_X_reSamp, train_comb_y_reSamp = resampler.fit_resample(X,y)



                train_comb = pd.DataFrame(train_comb_X_reSamp,columns=["F1","F2"])
                train_comb["Labels"] = train_comb_y_reSamp

                #store this instance of training data
                trainDataStore[name] = train_comb.copy()


            return trainDataStore, W, comp
        
        elif ExtractionType == "idle_motionIntent":
            X = train_idle_motionIntent_comb.iloc[:,:nCols-2]
            y = train_idle_motionIntent_comb.iloc[:,nCols-2]
            

            train_idle_motionIntent_comb = pd.DataFrame(X,columns=["F1","F2"])
            train_idle_motionIntent_comb["Labels"] = y

            return train_idle_motionIntent_comb, W_idleMotionIntentTranformationMatrix, comp


    else:

        if ExtractionType is None:

            #choosing not to balance data as this is more representative of the evaluation data and a possible real-world
            #application where the decision of left or right cannot be expected to be balanced

            #I can shuffle data since trials can be considered as indepedent samples with no temporal relationship
            X = train_comb.iloc[:,:nCols-2]
            y = train_comb.iloc[:,nCols-2]

        
            train_comb_X, test_comb_X, train_comb_y,test_comb_y = train_test_split(X,y,train_size=split_percentage,random_state=42)
            
            #samples may be heavily skewed towards one class in the training data (class imbalance)
            #We must resample the calibration data


            trainDataStore = {} #Stores different training data for each resampling method

            ######################################################## RESAMPLING ################################################################# 

            for name,resampler in [("RO",RandomOverSampler(random_state=42)),
                                    ("SMOTE",SMOTE(random_state=42)),
                                    ("RU",RandomUnderSampler(random_state=42)),
                                    ("CC",ClusterCentroids(random_state=42)),
                                    ("None",None)]:
                
                
                
                if not resampler:
                    train_comb = pd.DataFrame(train_comb_X,columns=["F1","F2"])
                    train_comb["Labels"] = train_comb_y

                    #store this instance of training data
                    trainDataStore[name] = train_comb.copy()
                    continue
                
                #Resample training data only 
                train_comb_X_reSamp, train_comb_y_reSamp = resampler.fit_resample(train_comb_X,train_comb_y)
                
                

                train_comb = pd.DataFrame(train_comb_X_reSamp,columns=["F1","F2"])
                train_comb["Labels"] = train_comb_y_reSamp
                
                #store this instance of training data 
                trainDataStore[name] = train_comb.copy()

            test_comb = pd.DataFrame(test_comb_X,columns=["F1","F2"])
            test_comb["Labels"] = test_comb_y

            return trainDataStore, test_comb, W, comp
        
        elif ExtractionType == "idle_motionIntent":

            X = train_idle_motionIntent_comb.iloc[:,:nCols-2]
            y = train_idle_motionIntent_comb.iloc[:,nCols-2]

            train_idle_motionIntent_comb_X, test_idle_motionIntent_comb_X, train_idle_motionIntent_comb_y,test_idle_motionIntent_comb_y = train_test_split(X,y,train_size=split_percentage,random_state=42)
            
            train_idle_motionIntent_comb = pd.DataFrame(train_idle_motionIntent_comb_X,columns=["F1","F2"])
            train_idle_motionIntent_comb["Labels"] = train_idle_motionIntent_comb_y

            test_idle_motionIntent_comb = pd.DataFrame(test_idle_motionIntent_comb_X,columns=["F1","F2"])
            test_idle_motionIntent_comb["Labels"] = test_idle_motionIntent_comb_y

            return train_idle_motionIntent_comb, test_idle_motionIntent_comb, W_idleMotionIntentTranformationMatrix, comp
    


In [17]:

import warnings
warnings.filterwarnings('ignore')

def testClassifier(rawData,trialWinStart,trialWinEnd,percentSplit,b=None,a=None,selectedChannels=None,resampler=None):

    """ Extract the best classifier performace from training data """
    
    #It may not yield the best results testing this classifier on data with a different trial 
    # width or start time since the data being tested is extracted using a different decomposition matrix (W) from the 
    # CSP algorithm. As the classifier being tested was initially fit to data extracted using a different decompostion matrix, 
    # poor performance is a likely result. It may be that in order to test the effect of increasing or decreasing the trial window 
    # length or the trial window start point, entirely new classifier tuning is required for each new window.
    
    if percentSplit < 1:
    
        trainDataStore, testData, W, _ = featureExtraction(rawData,percentSplit,
                                                        selectedChannels=selectedChannels,
                                                        trialWinStart=trialWinStart,
                                                        trialWinEnd=trialWinEnd,
                                                        b = b,
                                                        a = a)
    else:
        trainDataStore,  W, _ = featureExtraction(rawData,percentSplit,
                                                        selectedChannels=selectedChannels,
                                                        trialWinStart=trialWinStart,
                                                        trialWinEnd=trialWinEnd,
                                                        b = b,
                                                        a = a)
    

    trainData = trainDataStore[resampler] if resampler is not None else trainDataStore["None"]
    
    
    nCols = len(trainData.columns)
    ########################################## CLASSIFIER TUNING ####################################################

    bestModels = tuneModelHyperParams(trainData.iloc[:,:nCols-1],
                                      trainData.iloc[:,nCols-1],
                                      testData.iloc[:,:nCols-1],
                                      testData.iloc[:,nCols-1])
    

    bestModel, bestPerformance = None, 0

    for metric in bestModels:
        for model in bestModels[metric]:
            mod, perf = bestModels[metric][model]

            if perf > bestPerformance:
                bestMetric, bestModel, bestPerformance = metric, mod, perf

    return bestPerformance, bestModel, W, bestMetric, bestModels



In [18]:
def startRunningClassifier(evalData,calibratedModel,idleStateClassifier,comp, W_train, W_idle_motionIntent, trialWinStart,trialWinEnd,b=None, a=None):
    """ Extract class probabilities from evaluation test data for each time sample"""
    
    #Extracting Sampling Rate
    sRate = evalData["nfo"][0][0][0][0][0]

    evalEEG = evalData["cnt"].T 
   
    nSamples = evalEEG.shape[1] 

    trialWindow = np.arange(int(trialWinStart*sRate),int(trialWinEnd*sRate))

    preds = []

    features = []

    #Extract features first
    try:
        n = 0
        for i in range(nSamples):
            if n == 50000:
                break
            #capture signal in sliding window
            captured_signal = evalEEG[:,i+trialWindow]

            ### CHECK WHETHER SIGNAL IS IDLE STATE OR MOTION INTENT STATE #### 

            #filter captured signal
            nqfreq = 0.5*sRate
            if b is None and a is None:
                b , a = signal.iirfilter(6,[8/nqfreq,12/nqfreq])

            filtered_idle_motionIntent_test_trial = signal.filtfilt(b,a,captured_signal,axis=1)

            #Extract features from transformed data
            spatialFilters_idle_motionIntent = W_idle_motionIntent.T.dot(filtered_idle_motionIntent_test_trial)
            spatialFilters_idle_motionIntent = spatialFilters_idle_motionIntent[comp,:]

            feature_idle_motionIntent = np.log(np.var(spatialFilters_idle_motionIntent,axis=1)).T

            #check result if idleState classifier
            #print(idleStateClassifier.predict_proba(pd.DataFrame([feature_idle_motionIntent],columns=["F1","F2"])))
            idle_motionIntent_state = idleStateClassifier.predict(pd.DataFrame([feature_idle_motionIntent],columns=["F1","F2"]))

            if idle_motionIntent_state[0] == 1:
                if b is None and a is None:
                    b , a = signal.iirfilter(6,[8/nqfreq,12/nqfreq])

                filtered_test_trial = signal.filtfilt(b,a,captured_signal,axis=1)

                #Extract features from transformed data
                spatialFilters = W_train.T.dot(filtered_test_trial)

                spatialFilters = spatialFilters[comp,:]

                feature = np.log(np.var(spatialFilters,axis=1)).T

                features.append(feature)
            
               
            else:
                features.append(np.array([0,0]))

            n += 1

    
    except IndexError:
        pass

  
   
    for f in features:
      
        if f[0] == f[1] == 0:
            preds.extend([[0,0]])
            continue

        classProbabilities = calibratedModel.predict_proba(pd.DataFrame([f],columns=["F1","F2"]))
    
        preds.extend(classProbabilities)       

  
    
    return preds


In [19]:
def createFilterBank(fs):
    """
        Parameters
        ----------
        fs: Sampling frequency

        Returns
        -------
        filterBank: A dictionary with keys represented by a frequency band; (lower bound, upper bound)
                    and values as numerator (b) and denominator (a) coefficients of the iirfilter
    """
    
    filterBank = {}

    freq_bands = [(8,12),(8,15),(13,30)]

    for lowcut, highcut in freq_bands:
        nqfreq = 0.5*fs
        b , a = signal.iirfilter(4,[lowcut/nqfreq,highcut/nqfreq])
        filterBank[(lowcut,highcut)] = (b,a)

    return filterBank


In [20]:
def testWindowLength(data,modelNames,split):

    """ Extract best performance from trial window length tuning """
    
    bestTime = None
    bestPerformance = 0
    bestEstimator = None

    
    for model in modelNames:
        
        accVals = {}
        bestModelPerformance = 0
        bestModelEstimator = None
        bestModelTime = None

        for i in np.linspace(1,5,9):
            accVals[i-0.5], estimator, W = testClassifier(data,0.5,i,split,model)
            
            if accVals[i-0.5] > bestModelPerformance:
                bestModelPerformance = accVals[i-0.5]
                bestModelEstimator = estimator
                bestModelTime = i-0.5

        if bestModelPerformance > bestPerformance:
            bestPerformance = bestModelPerformance
            bestEstimator = bestModelEstimator
            bestTime = bestModelTime

        #plot acc Val
        x, y = zip(*accVals.items())
        plt.plot(x,y,label=f"{model}({bestModelPerformance})")
    
    plt.axvline(x=bestTime, color='purple', ls='--', lw=1.5, label=f"Best window length ({bestTime})")
    plt.legend()
    

    return bestEstimator, W
    


In [21]:
def testWindowStartTime(data,split,bestWindowEnd,times=None):

    """ Extract best performance from trial window start time tuning """

    bestTime = None
    bestPerformance = 0
    bestEstimator = None

    accVals = {}
    
    times = np.arange(0,2.1,0.1) if times is None else times

    for i in times:
        accVals[i], estimator, W , _ = testClassifier(data,i,bestWindowEnd,split)

        if accVals[i] > bestPerformance:
            bestPerformance = accVals[i]
            bestEstimator = estimator
            bestTime = i


        x, y = zip(*accVals.items())
        plt.plot(x,y,label=f"{type(estimator).__name__}({accVals[i]})")
   
    plt.axvline(x=bestTime, color='purple', ls='--', lw=1.5, label=f"Best start time({bestTime})")
    plt.legend()
    
    return bestEstimator, W
        


In [22]:
def testWindowStartTime_Length(data,modelNames,split):

    bestStartTimeLengthCombination = None
    bestPerformance = 0
    bestEstimator = None

    for model in modelNames:
        
        accVals = {}
        bestModelPerformance = 0
        bestModelEstimator = None
        bestModelStartTimeLengthCombination = None

        for windowStartTime in np.arange(0,2.1,0.1):
            for windowLength in np.linspace(1,5,9):
                accVals[(windowStartTime,windowLength)], estimator, W = testClassifier(data,
                                                                                       windowStartTime,
                                                                                       windowStartTime + windowLength,
                                                                                       split,
                                                                                       model)
                
                if accVals[windowStartTime,windowLength] > bestModelPerformance:
                    bestModelPerformance = accVals[windowStartTime,windowLength]
                    bestModelEstimator = estimator
                    bestModelStartTimeLengthCombination = (windowStartTime, windowLength)

        if bestModelPerformance > bestPerformance:
            bestPerformance = bestModelPerformance
            bestEstimator = bestModelEstimator
            bestStartTimeLengthCombination = bestModelStartTimeLengthCombination 

        
    print(bestStartTimeLengthCombination)
    return bestEstimator, W
        
        



In [23]:
def testFrequencyBand(rawData,filterBank,split,selectedChannels=None, trialWinStart=None, trialWinEnd=None, resampler=None):

    """ Extract best performance from frequency band tuning """


    totalBestBandPerformance = 0
    bandFreqRes = {}
    bestEstimator = None
    
    bestBand = None
    
    for lower,upper in filterBank.keys():

        b, a = filterBank[lower,upper]
        
        bestBandPerformanceVal = 0
        bestBandEstimator = None
       
        bestBandPerformanceVal,  bestBandEstimator, W, bestBandPerformanceMetric, allBandMetricPerformances = testClassifier(rawData,trialWinStart,trialWinEnd,split,b=b,a=a,selectedChannels=selectedChannels, resampler=resampler)
        
        if bestBandPerformanceVal > totalBestBandPerformance:
            bestBand = lower,upper
            totalBestBandPerformance = bestBandPerformanceVal
            bestEstimator = bestBandEstimator

        bandFreqRes[lower,upper] = (b,a)
        print(f"{lower}-{upper} : {bestBandPerformanceVal} ({bestBandPerformanceMetric})")
        print()
        print(f"All Metrics: {allBandMetricPerformances}")
        print("************************")

    
    return bestBand, bestEstimator, bandFreqRes[bestBand], W
        
